In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from tqdm import tqdm_notebook 

from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_blobs

from tensorflow.keras.datasets import mnist

In [ ]:

class SigmoidNeuron:
    def __init__(self):
 
        self.w = None
        self.b = None

    def perceptron(self, x):
        
        return np.dot(x, self.w) + self.b

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def grad_w_mse(self, x, y):
        y_pred = self.sigmoid(self.perceptron(x))
        return (y_pred - y) * y_pred * (1 - y_pred) * x

    def grad_b_mse(self, x, y):
        y_pred = self.sigmoid(self.perceptron(x))
        return (y_pred - y) * y_pred * (1 - y_pred)

    def grad_w_ce(self, x, y):
        y_pred = self.sigmoid(self.perceptron(x))
        if y == 0:
            return y_pred * x
        else:  # y == 1
            return - (1 - y_pred) * x

    def grad_b_ce(self, x, y):
        y_pred = self.sigmoid(self.perceptron(x))
        if y == 0:
            return y_pred
        else:  # y == 1
            return - (1 - y_pred)

    def fit(self, X, Y, epochs=10, lr=0.1, loss_fn="mse"):
        
        self.w = np.random.randn(X.shape[1])
        self.b = 0

        
        loss_history = []                            # loss

        for epoch in range(epochs):
            total_dw = 0
            total_db = 0
            for i in range(len(X)):
                x = X[i]
                y = Y[i]
                if loss_fn == "mse":
                    total_dw += self.grad_w_mse(x, y)
                    total_db += self.grad_b_mse(x, y)
                elif loss_fn == "ce":
                    total_dw += self.grad_w_ce(x, y)
                    total_db += self.grad_b_ce(x, y)

            
            total_dw /= len(X)
            total_db /= len(X)

            
            self.w -= lr * total_dw
            self.b -= lr * total_db

            Y_pred = self.predict(X)
            if loss_fn == "mse":
                loss = np.mean((Y - Y_pred) ** 2)
            elif loss_fn == "ce":
          
                Y_pred = np.clip(Y_pred, 1e-10, 1-1e-10)
                loss = -np.mean(Y*np.log(Y_pred) + (1-Y)*np.log(1-Y_pred))

            loss_history.append(loss)
            print(f"Epoch data inga varum {epoch+1}, Loss evlo {loss:.4f}")

        
        plt.plot(loss_history)
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.show()

    def predict(self, X):
     
        return np.array([self.sigmoid(self.perceptron(x)) for x in X])


In [ ]:
my_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","yellow","green"])
np.random.seed(0)

In [ ]:
data, labels = make_blobs(n_samples=1000, centers=4, n_features=2, random_state=0)
print(data.shape, labels.shape)

In [ ]:
plt.scatter(data[:,0], data[:,1], c=labels, cmap=my_cmap)
plt.show()

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(data, labels, stratify=labels, random_state=0)
print(X_train.shape, X_val.shape)


In [ ]:
class FirstFFNetwork:
    def __init__(self):
        self.w1 = np.random.randn()
        self.w2 = np.random.randn()
        self.w3 = np.random.randn()
        self.w4 = np.random.randn()
        self.w5 = np.random.randn()
        self.w6 = np.random.randn()
        self.b1 = 0
        self.b2 = 0
        self.b3 = 0

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward_pass(self, x):
        x1, x2 = x
        a1 = self.w1*x1 + self.w2*x2 + self.b1
        h1 = self.sigmoid(a1)
        a2 = self.w3*x1 + self.w4*x2 + self.b2
        h2 = self.sigmoid(a2)
        a3 = self.w5*h1 + self.w6*h2 + self.b3
        h3 = self.sigmoid(a3)
        return h1, h2, h3

    def grad(self, x, y):
        x1, x2 = x
        h1, h2, h3 = self.forward_pass(x)
        d_output = (h3 - y) * h3 * (1 - h3)
        dw5 = d_output * h1
        dw6 = d_output * h2
        db3 = d_output
        d_h1 = d_output * self.w5 * h1 * (1 - h1)
        dw1 = d_h1 * x1
        dw2 = d_h1 * x2
        db1 = d_h1
        d_h2 = d_output * self.w6 * h2 * (1 - h2)
        dw3 = d_h2 * x1
        dw4 = d_h2 * x2
        db2 = d_h2
        return dw1, dw2, dw3, dw4, dw5, dw6, db1, db2, db3

    def fit(self, X, Y, epochs=10, lr=0.1, display_loss=False):
        loss_history = []

        for epoch in range(epochs):
            dw1 = dw2 = dw3 = dw4 = dw5 = dw6 = db1 = db2 = db3 = 0
            for x, y in zip(X, Y):
                g = self.grad(x, y)
                dw1 += g[0]; dw2 += g[1]; dw3 += g[2]; dw4 += g[3]
                dw5 += g[4]; dw6 += g[5]; db1 += g[6]; db2 += g[7]; db3 += g[8]

            m = len(X)
            self.w1 -= lr * dw1 / m
            self.w2 -= lr * dw2 / m
            self.w3 -= lr * dw3 / m
            self.w4 -= lr * dw4 / m
            self.w5 -= lr * dw5 / m
            self.w6 -= lr * dw6 / m
            self.b1 -= lr * db1 / m
            self.b2 -= lr * db2 / m
            self.b3 -= lr * db3 / m

            if display_loss:
                Y_pred = self.predict(X)
                loss = np.mean((Y - Y_pred) ** 2)
                loss_history.append(loss)
                print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

        if display_loss:
            plt.plot(loss_history)
            plt.xlabel("Epochs")
            plt.ylabel("MSE Loss")
            plt.show()

    def predict(self, X):
        return np.array([self.forward_pass(x)[2] for x in X])

In [ ]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])
Y = np.array([0,1,1,0])

In [ ]:
network = FirstFFNetwork()
network.fit(X, Y, epochs=5000, lr=0.5, display_loss=True)

In [ ]:
predictions = network.predict(X)
print("Predictions:", predictions)

In [ ]:
ffn = FirstFFNetwork()
ffn.fit(X, Y, epochs=2000, lr=0.01, display_loss=True)

In [ ]:
Y_pred_train = ffn.predict(X_train)
Y_pred_binarised_train = (Y_pred_train >= 0.5).astype("int").ravel()
Y_pred_val = ffn.predict(X_val)
Y_pred_binarised_val = (Y_pred_val >= 0.5).astype("int").ravel()
accuracy_train = accuracy_score(Y_pred_binarised_train, Y_train)
accuracy_val = accuracy_score(Y_pred_binarised_val, Y_val)

print("Training points", round(accuracy_train, 2))
print("Validation points", round(accuracy_val, 2))

In [ ]:

X = np.array([[0,0],[0,1],[1,0],[1,1]])
Y = np.array([0,1,1,0])


plt.scatter(X[:,0], X[:,1], c=Y, cmap='bwr')
plt.xlabel("X1")
plt.ylabel("X2")
plt.show()


In [ ]:
ffn = FirstFFNetwork()
ffn.fit(X, Y, epochs=5000, lr=0.5, display_loss=True)
preds_ffn = ffn.predict(X)
print("FF Network predictions:", preds_ffn)


In [ ]:
from tensorflow.keras.datasets import mnist


(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)


In [ ]:

X_train = X_train.reshape(X_train.shape[0], 28*28)
X_test = X_test.reshape(X_test.shape[0], 28*28)


In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0


In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, 10)  
y_test = to_categorical(y_test, 10)


In [ ]:

input_size = 784
hidden_size = 128
output_size = 10
learning_rate = 0.01

# Weights
W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))

W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))


In [ ]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / np.sum(e_x, axis=1, keepdims=True)


In [ ]:
#hidden
Z1 = np.dot(X_train, W1) + b1
A1 = relu(Z1)

#output
Z2 = np.dot(A1, W2) + b2
A2 = softmax(Z2)

print(A2)
print(A1)
print("a2",A2[0])

In [ ]:
def cross_entropy(y_true, y_pred):
    n_samples = y_true.shape[0]
    return -np.sum(y_true * np.log(y_pred + 1e-8)) / n_samples


In [ ]:

dZ2 = A2 - y_train            
dW2 = np.dot(A1.T, dZ2) / X_train.shape[0]
db2 = np.sum(dZ2, axis=0, keepdims=True) / X_train.shape[0]


dA1 = np.dot(dZ2, W2.T)
dZ1 = dA1 * relu_derivative(Z1)
dW1 = np.dot(X_train.T, dZ1) / X_train.shape[0]
db1 = np.sum(dZ1, axis=0, keepdims=True) / X_train.shape[0]


W1 -= learning_rate * dW1
b1 -= learning_rate * db1
W2 -= learning_rate * dW2
b2 -= learning_rate * db2

print(db2)


In [ ]:
np.argmax(A2[0])  


# custom data training


In [ ]:

train_data = pd.read_csv("D:/git/data/fashion-mnist_train.csv")
test_data = pd.read_csv("D:/git/data/fashion-mnist_test.csv")

X_train = train_data.drop("label", axis=1).values / 255.0
y_train = to_categorical(train_data["label"].values, 10)

X_test = test_data.drop("label", axis=1).values / 255.0
y_test = to_categorical(test_data["label"].values, 10)


In [ ]:
input_size = 784      # 28x28 pixels
hidden_size = 128
output_size = 10      # 10 classes
learning_rate = 0.1
epochs = 200


In [ ]:
W1 = np.random.randn(input_size, hidden_size) * 0.01
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size) * 0.01
b2 = np.zeros((1, output_size))


In [ ]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / np.sum(e_x, axis=1, keepdims=True)

def cross_entropy(y_true, y_pred):
    n_samples = y_true.shape[0]
    return -np.sum(y_true * np.log(y_pred + 1e-8)) / n_samples


In [ ]:
for epoch in range(epochs):
    
    Z1 = np.dot(X_train, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)


    loss = cross_entropy(y_train, A2)

   
    dZ2 = A2 - y_train
    dW2 = np.dot(A1.T, dZ2) / X_train.shape[0]
    db2 = np.sum(dZ2, axis=0, keepdims=True) / X_train.shape[0]

    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X_train.T, dZ1) / X_train.shape[0]
    db1 = np.sum(dZ1, axis=0, keepdims=True) / X_train.shape[0]

   
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")


In [ ]:
# Forward pass on test set
Z1_test = np.dot(X_test, W1) + b1
A1_test = relu(Z1_test)
Z2_test = np.dot(A1_test, W2) + b2
A2_test = softmax(Z2_test)

# Predicted labels
y_pred = np.argmax(A2_test, axis=1)
y_true = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred == y_true)
print(f"Test Accuracy: {accuracy*100:.2f}%")


In [ ]:

i = 0
plt.imshow(X_test[i].reshape(28,28), cmap='blue')
plt.show()
print("Predicted:", y_pred[i], "Actual:", y_true[i])
